In [1]:
import numpy as np

In [2]:
# Original (incorrect) EIF 

n = 10000
rng = np.random.default_rng()
z = rng.binomial(n=1,p=0.5, size=n)
x = (1/(1 + np.exp(-(z+rng.normal(size=n)))) >= 0.5).astype(np.int8)
w = (1/(1 + np.exp(-(x + z + rng.normal(size=n)))) >= 0.5).astype(np.int8)
y = (1/(1 + np.exp(-(x + rng.normal(size=n)))) >= 0.5).astype(np.int8)
#y = (1/(1 + np.exp(-(2*x +rng.normal(size=n)))) >= 0.5).astype(np.int8)

de = np.zeros(n)
ie = np.zeros(n)
se = np.zeros(n)
for i in range(n):
    
    p_xwz_1 = np.mean(np.all([x==1, w==w[i], z==z[i]], axis=0))
    p_xwz_0 = np.mean(np.all([x==0, w==w[i], z==z[i]], axis=0))
    p_xz_0 = np.mean(np.all([x==0, z==z[i]], axis=0))
    p_xz_1 = np.mean(np.all([x==1, z==z[i]], axis=0))

    p_x1 = np.mean([x==1])
    p_x0 = 1 - p_x1

    p_ycond_1 = np.mean(y[np.where(np.all([x==1, w==w[i], z==z[i]], axis=0))])
    p_ycond_0 = np.mean(y[np.where(np.all([x==0, w==w[i], z==z[i]], axis=0))])

    p_wcond_x1 = np.mean(w[np.where(np.all([x==1, z==z[i]], axis=0))])
    p_wcond_x0 = np.mean(w[np.where(np.all([x==0, z==z[i]], axis=0))])
    p_zcond_x0 = np.mean(z[np.where(np.all([x==0], axis=0))])
    p_zcond_x1 = np.mean(z[np.where(np.all([x==1], axis=0))])
    p_ycond_w1 = np.mean(y[np.where(np.all([x==0, w==1, z==z[i]], axis=0))])
    p_ycond_w0 = np.mean(y[np.where(np.all([x==0, w==0, z==z[i]], axis=0))])

    
 
    de[i] = ( ( (1/p_xwz_1)*((y[i]==1) - p_ycond_1)*p_wcond_x0*p_zcond_x0 
    - (1/p_xwz_0)*((y[i]==1) - p_ycond_0)*p_wcond_x0*p_zcond_x0  ) +
    (1/p_xz)*p_ycond_1*p_zcond_x0 - (1/p_xz)*p_ycond_0*p_zcond_x0) 

    de[i] += (p_ycond_1-p_ycond_0)*p_wcond_x0*p_zcond_x0

    ie[i] = ( (1/p_xwz_0)*((y[i]==1) - p_ycond_0)*p_wcond_x1*p_zcond_x0 - 
    (1/p_xwz_0)*((y[i]==1) - p_ycond_0)*p_wcond_x0*p_zcond_x0 )
    ie[i] += (1/p_xwz_1 - 1/p_xwz_0)*p_ycond_0*p_zcond_x0
    ie[i] += ( ( (p_xz_1 - p_xz_0)/(p_xz_1*p_x0) ) * 
        (p_wcond_x1*p_ycond_w1 + (1-p_wcond_x1)*p_ycond_w0) )

    #ie[i] += p_ycond_0*p_wcond_x1*p_zcond_x0 - p_ycond_0*p_wcond_x0*p_zcond_x0

    se[i] += ( (1/p_xwz_0)*((y[i]==1) - p_ycond_0)*p_wcond_x0*p_zcond_x1 - 
    (1/p_xwz_0)*((y[i]==1) - p_ycond_0)*p_wcond_x0*p_zcond_x0 )
    se[i] += (1/p_xwz_0)*p_ycond_0*(p_zcond_x1 - p_zcond_x0)
    se[i] += ( ( (p_xz_0 - p_xz_1)/(p_xz_0*p_x1) ) * 
        (p_wcond_x0*p_ycond_w1 + (1-p_wcond_x0)*p_ycond_w0) )

    #se[i] = p_ycond_0*p_wcond_x0*p_zcond_x1 - p_ycond_0*p_wcond_x0*p_zcond_x0

print(np.mean(de))
print(np.mean(ie))
print(np.mean(se))

NameError: name 'p_xz' is not defined

In [3]:
# Corrected EIF - Plug-in
n = 1000
rng = np.random.default_rng()
z = rng.binomial(n=1,p=0.5, size=2*n)
x = (1/(1 + np.exp(-(0.1*z+rng.normal(size=2*n)))) >= 0.5).astype(np.int8)
w = (1/(1 + np.exp(-(0.1*x + 0.1*z + rng.normal(size=2*n)))) >= 0.5).astype(np.int8)
y = (1/(1 + np.exp(-(5*x +  rng.normal(size=2*n)))) >= 0.5).astype(np.int8)
#y = (1/(1 + np.exp(-(2*x +rng.normal(size=n)))) >= 0.5).astype(np.int8)

ztr, zte = z[:n], z[n:]
xtr, xte = x[:n], x[n:]
wtr, wte = w[:n], w[n:]
ytr, yte = y[:n], y[n:]

de = np.zeros(n)
ie = np.zeros(n)
se = np.zeros(n)

de_plug = np.zeros(n)
ie_plug = np.zeros(n)
se_plug = np.zeros(n)

for i in range(n):
    p_ycond_0 = np.mean(
        ytr[np.where(
            np.all([wtr==wte[i], ztr==zte[i], xtr==0],axis=0)
            )]
            )
    p_ycond_1 = np.mean(
        ytr[np.where(
            np.all([wtr==wte[i], ztr==zte[i], xtr==1],axis=0)
            )]
            )
    
    p_wcond_1 = np.mean(
        wtr[np.where(
            np.all([ztr==zte[i], xtr==1],axis=0)
        )] 
    )
    p_wcond_0 = np.mean(
        wtr[np.where(
            np.all([ztr==zte[i], xtr==0],axis=0)
        )] 
    )

    p_zcond_1 = np.mean(
        ztr[np.where(np.all([xtr==1],axis=0))]
    )
    p_zcond_0 = np.mean(
        ztr[np.where(np.all([xtr==0],axis=0))]
    )

    p_xwz_1 = np.mean(np.all([xtr==1, wtr==wte[i], ztr==zte[i]], axis=0))
    p_xwz_0 = np.mean(np.all([xtr==0, wtr==wte[i], ztr==zte[i]], axis=0))
    p_xz_0 = np.mean(np.all([xtr==0, ztr==zte[i]], axis=0))
    p_xz_1 = np.mean(np.all([xtr==1, ztr==zte[i]], axis=0))

    p_x1 = np.mean([xtr==1])
    p_x0 = 1 - p_x1

    de[i] = ( (xte[i]==1)/p_xwz_1 * ((yte[i]==1) - p_ycond_1)*p_wcond_0*p_zcond_0 +
        (xte[i]==0)/p_xz_0 * ((wte[i]==1)-p_wcond_0)*p_ycond_1*p_zcond_0 + 
        (xte[i]==0)/p_xz_0 * ((wte[i]==0)-(1-p_wcond_0))*p_ycond_1*p_zcond_0 + 
        (xte[i]==0)/p_x0 * ((zte[i]==1)-p_zcond_0)*p_ycond_1*p_wcond_0 +
        (xte[i]==0)/p_x0 * ((zte[i]==0)-(1-p_zcond_0))*p_ycond_1*p_wcond_0 )
    de[i] -= ( (xte[i]==0)/p_xwz_0 * ((yte[i]==1) - p_ycond_0)*p_wcond_0*p_zcond_0 +
        (xte[i]==0)/p_xz_0 * ((wte[i]==1)-p_wcond_0)*p_ycond_0*p_zcond_0 + 
        (xte[i]==0)/p_xz_0 * ((wte[i]==0)-(1-p_wcond_0))*p_ycond_0*p_zcond_0 + 
        (xte[i]==0)/p_x0 * ((zte[i]==1)-p_zcond_0)*p_ycond_0*p_wcond_0 +
        (xte[i]==0)/p_x0 * ((zte[i]==0)-(1-p_zcond_0))*p_ycond_0*p_wcond_0 )
    de_plug[i] += (p_ycond_1-p_ycond_0)*p_wcond_0*p_zcond_0

    ie[i] = ( (xte[i]==0)/p_xwz_0 * ((yte[i]==1) - p_ycond_0)*p_wcond_1*p_zcond_0 +
        (xte[i]==1)/p_xz_1 * ((wte[i]==1)-p_wcond_1)*p_ycond_0*p_zcond_0 + 
        (xte[i]==1)/p_xz_1 * ((wte[i]==0)-(1-p_wcond_1))*p_ycond_0*p_zcond_0 + 
        (xte[i]==0)/p_x0 * ((zte[i]==1)-p_zcond_0)*p_ycond_0*p_wcond_1 +
        (xte[i]==0)/p_x0 * ((zte[i]==0)-(1-p_zcond_0))*p_ycond_0*p_wcond_1 )
    ie[i] -= ( (xte[i]==0)/p_xwz_0 * ((yte[i]==1) - p_ycond_0)*p_wcond_0*p_zcond_0 +
        (xte[i]==0)/p_xz_0 * ((wte[i]==1)-p_wcond_0)*p_ycond_0*p_zcond_0 + 
        (xte[i]==0)/p_xz_0 * ((wte[i]==0)-(1-p_wcond_0))*p_ycond_0*p_zcond_0 + 
        (xte[i]==0)/p_x0 * ((zte[i]==1)-p_zcond_0)*p_ycond_0*p_wcond_0 +
        (xte[i]==0)/p_x0 * ((zte[i]==0)-(1-p_zcond_0))*p_ycond_0*p_wcond_0 )
    ie_plug[i] += p_ycond_0*(p_wcond_1 - p_wcond_0)*p_zcond_0

    se[i] = ( (xte[i]==0)/p_xwz_0 * ((yte[i]==1) - p_ycond_0)*p_wcond_0*p_zcond_1 +
        (xte[i]==0)/p_xz_0 * ((wte[i]==1)-p_wcond_0)*p_ycond_0*p_zcond_1 + 
        (xte[i]==0)/p_xz_0 * ((wte[i]==0)-(1-p_wcond_0))*p_ycond_0*p_zcond_1 + 
        (xte[i]==1)/p_x1 * ((zte[i]==1)-p_zcond_1)*p_ycond_0*p_wcond_0 +
        (xte[i]==1)/p_x1 * ((zte[i]==0)-(1-p_zcond_1))*p_ycond_0*p_wcond_0 )
    se[i] -= ( (xte[i]==0)/p_xwz_0 * ((yte[i]==1) - p_ycond_0)*p_wcond_0*p_zcond_0 +
        (xte[i]==0)/p_xz_0 * ((wte[i]==1)-p_wcond_0)*p_ycond_0*p_zcond_0 + 
        (xte[i]==0)/p_xz_0 * ((wte[i]==0)-(1-p_wcond_0))*p_ycond_0*p_zcond_0 + 
        (xte[i]==0)/p_x0 * ((zte[i]==1)-p_zcond_0)*p_ycond_0*p_wcond_0 +
        (xte[i]==0)/p_x0 * ((zte[i]==0)-(1-p_zcond_0))*p_ycond_0*p_wcond_0 )
    se_plug[i] += p_ycond_0*p_wcond_0*(p_zcond_1-p_zcond_0)
    

print(np.mean(de))
print(np.mean(ie))
print(np.mean(se))
print('Plug in estiamtes')
print(np.mean(de_plug))
print(np.mean(ie_plug))
print(np.mean(se_plug))

print('Variance estimates')
print(np.var(de))
print(np.var(ie))
print(np.var(se))

-0.0075220749214117266
0.002502395671524035
0.0009382055002459938
Plug in estiamtes
0.11848926084542771
0.014096774455543267
0.014167645144328557
Variance estimates
0.47080949511523557
0.008022878034259854
0.007324295417313286


In [4]:
# Corrected EIF - Ground Truth

n = 1000
rng = np.random.default_rng()
z = rng.binomial(n=1,p=0.5, size=2*n)
x_cond = 1/(1 + np.exp(-(0.1*z+rng.normal(size=2*n))))
x = (x_cond >= 0.5).astype(np.int8)
px = (x==1)*np.mean(x) + (x==0)*(1-np.mean(x))
z_cond = x_cond * 0.5 / px
w_cond = 1/(1 + np.exp(-(0.1*x + 0.1*z + rng.normal(size=2*n))))
w = (w_cond >= 0.5).astype(np.int8)
y_cond = 1/(1 + np.exp(-(5*x + z +  rng.normal(size=2*n))))
y = (y_cond >= 0.5).astype(np.int8)
#y = (1/(1 + np.exp(-(2*x +rng.normal(size=n)))) >= 0.5).astype(np.int8)

ztr, zte = z[:n], z[n:]
xtr, xte = x[:n], x[n:]
wtr, wte = w[:n], w[n:]
ytr, yte = y[:n], y[n:]

de = np.zeros(n)
ie = np.zeros(n)
se = np.zeros(n)

de_plug = np.zeros(n)
ie_plug = np.zeros(n)
se_plug = np.zeros(n)

for i in range(n):
    p_ycond_0 = np.mean(
        y_cond[np.where(
            np.all([wtr==wte[i], ztr==zte[i], xtr==0],axis=0)
            )]
            )
    p_ycond_1 = np.mean(
        y_cond[np.where(
            np.all([wtr==wte[i], ztr==zte[i], xtr==1],axis=0)
            )]
            )
    
    p_wcond_1 = np.mean(
        w_cond[np.where(
            np.all([ztr==zte[i], xtr==1],axis=0)
        )] 
    )
    p_wcond_0 = np.mean(
        w_cond[np.where(
            np.all([ztr==zte[i], xtr==0],axis=0)
        )] 
    )

    p_zcond_1 = np.mean(
        z_cond[np.where(np.all([xtr==1],axis=0))]
    )
    p_zcond_0 = np.mean(
        z_cond[np.where(np.all([xtr==0],axis=0))]
    )

    p_xwz_1 = np.mean(np.all([xtr==1, wtr==wte[i], ztr==zte[i]], axis=0))
    p_xwz_0 = np.mean(np.all([xtr==0, wtr==wte[i], ztr==zte[i]], axis=0))
    p_xz_0 = np.mean(np.all([xtr==0, ztr==zte[i]], axis=0))
    p_xz_1 = np.mean(np.all([xtr==1, ztr==zte[i]], axis=0))

    p_x1 = np.mean([xtr==1])
    p_x0 = 1 - p_x1

    de[i] = ( (xte[i]==1)/p_xwz_1 * ((yte[i]==1) - p_ycond_1)*p_wcond_0*p_zcond_0 +
        (xte[i]==0)/p_xz_0 * ((wte[i]==1)-p_wcond_0)*p_ycond_1*p_zcond_0 + 
        (xte[i]==0)/p_xz_0 * ((wte[i]==0)-(1-p_wcond_0))*p_ycond_1*p_zcond_0 + 
        (xte[i]==0)/p_x0 * ((zte[i]==1)-p_zcond_0)*p_ycond_1*p_wcond_0 +
        (xte[i]==0)/p_x0 * ((zte[i]==0)-(1-p_zcond_0))*p_ycond_1*p_wcond_0 )
    de[i] -= ( (xte[i]==0)/p_xwz_0 * ((yte[i]==1) - p_ycond_0)*p_wcond_0*p_zcond_0 +
        (xte[i]==0)/p_xz_0 * ((wte[i]==1)-p_wcond_0)*p_ycond_0*p_zcond_0 + 
        (xte[i]==0)/p_xz_0 * ((wte[i]==0)-(1-p_wcond_0))*p_ycond_0*p_zcond_0 + 
        (xte[i]==0)/p_x0 * ((zte[i]==1)-p_zcond_0)*p_ycond_0*p_wcond_0 +
        (xte[i]==0)/p_x0 * ((zte[i]==0)-(1-p_zcond_0))*p_ycond_0*p_wcond_0 )
    de_plug[i] += (p_ycond_1-p_ycond_0)*p_wcond_0*p_zcond_0

    ie[i] = ( (xte[i]==0)/p_xwz_0 * ((yte[i]==1) - p_ycond_0)*p_wcond_1*p_zcond_0 +
        (xte[i]==1)/p_xz_1 * ((wte[i]==1)-p_wcond_1)*p_ycond_0*p_zcond_0 + 
        (xte[i]==1)/p_xz_1 * ((wte[i]==0)-(1-p_wcond_1))*p_ycond_0*p_zcond_0 + 
        (xte[i]==0)/p_x0 * ((zte[i]==1)-p_zcond_0)*p_ycond_0*p_wcond_1 +
        (xte[i]==0)/p_x0 * ((zte[i]==0)-(1-p_zcond_0))*p_ycond_0*p_wcond_1 )
    ie[i] -= ( (xte[i]==0)/p_xwz_0 * ((yte[i]==1) - p_ycond_0)*p_wcond_0*p_zcond_0 +
        (xte[i]==0)/p_xz_0 * ((wte[i]==1)-p_wcond_0)*p_ycond_0*p_zcond_0 + 
        (xte[i]==0)/p_xz_0 * ((wte[i]==0)-(1-p_wcond_0))*p_ycond_0*p_zcond_0 + 
        (xte[i]==0)/p_x0 * ((zte[i]==1)-p_zcond_0)*p_ycond_0*p_wcond_0 +
        (xte[i]==0)/p_x0 * ((zte[i]==0)-(1-p_zcond_0))*p_ycond_0*p_wcond_0 )
    ie_plug[i] += p_ycond_0*(p_wcond_1 - p_wcond_0)*p_zcond_0

    se[i] = ( (xte[i]==0)/p_xwz_0 * ((yte[i]==1) - p_ycond_0)*p_wcond_0*p_zcond_1 +
        (xte[i]==0)/p_xz_0 * ((wte[i]==1)-p_wcond_0)*p_ycond_0*p_zcond_1 + 
        (xte[i]==0)/p_xz_0 * ((wte[i]==0)-(1-p_wcond_0))*p_ycond_0*p_zcond_1 + 
        (xte[i]==1)/p_x1 * ((zte[i]==1)-p_zcond_1)*p_ycond_0*p_wcond_0 +
        (xte[i]==1)/p_x1 * ((zte[i]==0)-(1-p_zcond_1))*p_ycond_0*p_wcond_0 )
    se[i] -= ( (xte[i]==0)/p_xwz_0 * ((yte[i]==1) - p_ycond_0)*p_wcond_0*p_zcond_0 +
        (xte[i]==0)/p_xz_0 * ((wte[i]==1)-p_wcond_0)*p_ycond_0*p_zcond_0 + 
        (xte[i]==0)/p_xz_0 * ((wte[i]==0)-(1-p_wcond_0))*p_ycond_0*p_zcond_0 + 
        (xte[i]==0)/p_x0 * ((zte[i]==1)-p_zcond_0)*p_ycond_0*p_wcond_0 +
        (xte[i]==0)/p_x0 * ((zte[i]==0)-(1-p_zcond_0))*p_ycond_0*p_wcond_0 )
    se_plug[i] += p_ycond_0*p_wcond_0*(p_zcond_1-p_zcond_0)
    

print(np.mean(de))
print(np.mean(ie))
print(np.mean(se))
print('Plug in estiamtes')
print(np.mean(de_plug))
print(np.mean(ie_plug))
print(np.mean(se_plug))

print('Variance estimates')
print(np.var(de))
print(np.var(ie))
print(np.var(se))

-0.07065181818598638
0.003214445451044218
0.06139720877830179
Plug in estiamtes
0.07154800724879817
0.005365748496415346
0.0874757126537377
Variance estimates
0.23424162922783884
0.0008399019151143591
0.15446293300578942


In [28]:
print(np.mean(y_cond[np.where(x==1)]))
np.mean(y_cond[np.where(x==0)])

0.9894846940385302


0.5068879063859981

In [175]:
print( np.mean(
        ytr[np.where(
            np.all([wtr==0, ztr==0, xtr==0],axis=0)
            )]
            )
)
print( np.mean(
        ytr[np.where(
            np.all([wtr==1, ztr==0, xtr==0],axis=0)
            )]
            )
)
print( np.mean(
        ytr[np.where(
            np.all([wtr==0, ztr==0, xtr==1],axis=0)
            )]
            )
)
print( np.mean(
        ytr[np.where(
            np.all([wtr==0, ztr==1, xtr==1],axis=0)
            )]
            )
)
print( np.mean(
        ytr[np.where(
            np.all([wtr==1, ztr==1, xtr==1],axis=0)
            )]
            )
)

0.41346153846153844
0.5126050420168067
0.8604651162790697
0.9
0.8329466357308585


In [172]:
print( np.mean(
        wtr[np.where(
            np.all([ztr==0, xtr==0],axis=0)
        )] 
    )
)
print( np.mean(
        wtr[np.where(
            np.all([ztr==0, xtr==1],axis=0)
        )] 
    )
)
print( np.mean(
        wtr[np.where(
            np.all([ztr==1, xtr==0],axis=0)
        )] 
    )
)
print( np.mean(
        wtr[np.where(
            np.all([ztr==1, xtr==1],axis=0)
        )] 
    )
)

0.5031201248049922
0.8442723196086425
0.8644295302013423
0.976875884851345
